3 atividade: trabalho de determinação de parâmetros exoplanetários utilizando dados de velocidaderadial e de trânsito;

Quais parametros estudar:

Determinação de massa, raio, semi eixo maior, período e outros parâmetros (se necessário) do planeta escolhido;

Estudo da zona de habitabilidade, temperatura de equilíbrio, grau de insolação do planeta (comparação com o sistema solar e com outros exoplanetas);

Estudo de densidade planetária com base no valor obtido (comparação com o sistema solar e com outros exoplanetas);

Método:

Escolha entre 2 e três planetas que possuam, necessariamente, dados de profundidade de trânsito (Transit Depth[%]), amplitude da variação da velocidade (Radial Velocity Amplitude [m/s]) e período de velocidade radial (Orbital Period [days]);

Lembrem de descrever os telescópios e satélites que fizeram as obtenções dos dados;Acesse o catálogo da NASA para fazer a seleção dos planetas:

https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=planets

https://exoplanetarchive.ipac.caltech.edu/

Em anexo está a apostila do estudo de exoplanetas. Nela vocês vão encontrar todas as equações necessárias assim como a discussão para o desenvolvimento do trabalho.

Os Planetas Escolhidos foram BD+20 594b (BD+20 594), CoRoT-2b (CoRoT-2), HAT-P-14b (HAT-P-14)

In [56]:
class Conversions:
    def __init__(self, daysInSeconds, auInMeter, year, sunRatio, sunMass, jupMass, jupRatio):
        self.DAYS_IN_SECONDS = daysInSeconds
        self.AU_IN_METER = auInMeter
        self.YEAR = year
        self.SUN_RATIO = sunRatio
        self.SUN_MASS = sunMass
        self.JUP_MASS = jupMass
        self.JUP_RATIO = jupRatio
        
    def convert_DaysToSeconds(self, days):
        return days*self.DAYS_IN_SECONDS
    
    def convert_AuToMeter(self, au):
        return au*self.AU_IN_METER
    
    def convertDaysToYear(self, period):
        return period/self.YEAR
    
    def convertStellarRatioToMeter(self, stellarRatio):
        return stellarRatio*self.SUN_RATIO
    
    def convertStellarMassToKg(self, stellarMass):
        return stellarMass*self.SUN_MASS
    
    def convertJupRatioToMeter(self, planetRatio):
        return planetRatio*self.JUP_RATIO
    
    def convertJupMassToKg(self, planetMass):
        return planetMass*self.JUP_MASS

In [132]:
import math

class ExoplanetParameters:
    
    def __init__(self, solTemp, albedo, gravitCTE): 
        self.conversions = Conversions(86400, 149597870700, 365, 695500000, 1.989*math.pow(10,30), 1.898*math.pow(10,27), 69.911*math.pow(10,3))
        self.SOLAR_TEMPERATURE = solTemp
        self.ALBEDO = albedo
        self.GRAVITY_CTE = gravitCTE
        
    def sunFlowRadiation(self):
        return 3.846*math.pow(10,26)/(4*math.pi*math.pow(149597870700, 2))
    
    def radiationFlow(self, stellarRatio_inSolarRatio, effectiveTemp, semiMajorAxis):
        return math.pow(stellarRatio_inSolarRatio, 2)*math.pow(effectiveTemp/self.SOLAR_TEMPERATURE, 4)*math.pow(1/semiMajorAxis, 2)
    
    def semiMajorAxis_Kepler(self, orbitalPeriod, starMass, planetMass):
        num = math.pow(self.conversions.convert_DaysToSeconds(orbitalPeriod), 2)*self.GRAVITY_CTE*(self.conversions.convertStellarMassToKg(starMass)+self.conversions.convertJupMassToKg(planetMass))
        return math.pow(num/(4*math.pow(math.pi, 2)), 1/3)/149597870700
    
    def semiMajorAxis_Bar(self, orbitalPeriod, starMass, planetMass):
        num = math.pow(self.conversions.convert_DaysToSeconds(orbitalPeriod), 2)*self.GRAVITY_CTE*math.pow(self.conversions.convertJupMassToKg(planetMass),3)
        den = 4*math.pow(math.pi, 2)*math.pow(self.conversions.convertStellarMassToKg(starMass)+self.conversions.convertJupMassToKg(planetMass),2)
        return math.pow(num/den,1/3)/149597870700
    
    def equilibriumTemperature(self, effectiveTemp, semiMajorAxis, stellarRatio):
        return effectiveTemp*math.pow(1-self.ALBEDO, 1/4)*math.pow((stellarRatio/(2*self.ALBEDO)), 1/2)
            
    def habitabilityZone(self, stellarLuminosity_inSolarLuminosity):
        intern = 0.75*math.pow(stellarLuminosity_inSolarLuminosity, 2)
        central = math.pow(stellarLuminosity_inSolarLuminosity, 2)
        extern = 1.77*math.pow(stellarLuminosity_inSolarLuminosity, 2)
        return (intern, central, extern)
    
    def planetRadius(self, transitDepth, stellarRatio_inSolarRatio):
        return math.pow(transitDepth*1.049, 1/2)*(stellarRatio_inSolarRatio)
    
    def planetMass_inJupMass(self, K, orbitalPeriod, starMass):
        return (K/28.4)*math.pow(self.conversions.convertDaysToYear(orbitalPeriod), 1/3)*math.pow(starMass ,2/3)
    
    def transitProbability(self, starRatio, planetRatio, semiMajorAxis, eccentricity):
        num = self.conversions.convertJupRatioToMeter(planetRatio)+self.conversions.convertStellarRatioToMeter(starRatio)
        den = self.conversions.convert_AuToMeter(semiMajorAxis)*(1+math.pow(eccentricity, 2))
        return num/den
    
    def transitTime(self, starRatio, planetRatio, orbitalPeriod, semiMajorAxis):
        starRatio_m = self.conversions.convertStellarRatioToMeter(starRatio)
        planetRatio_m = self.conversions.convertJupRatioToMeter(planetRatio)
        orbPeriod_seconds = self.conversions.convert_DaysToSeconds(orbitalPeriod)
        semiMajorAxis_m = self.conversions.convert_AuToMeter(semiMajorAxis)
        
        a = math.pow(1+(planetRatio_m/starRatio_m),2)
        b = math.pow(semiMajorAxis_m/starRatio_m, 2)
        c = orbPeriod_seconds/math.pi
        return c*math.asin((starRatio_m/semiMajorAxis_m)*math.pow(a ,1/2))/3600

In [134]:
import numpy as np
import math

exoParams = ExoplanetParameters(5777, 0.367, 6.674184*math.pow(10, -11))
print(exoParams.habitabilityZone(2))

planet_1 = {
    "name": "BD+20 594b",
    "transitDepth": 0.0490,
    "radialVelocityAmp": 3.1,
    "orbitalPeriod": 41.688644,
    "eccentricity": 0,
    "star":{
        "effectiveTemperature": 5651.760,
        "stellarRatio_inSolarRatio": 1.2449300,
        "stellarLuminosity": 0.15372068,
        "stellarMass_inSolarMass": 1.0030000
    }
}

planet_2 = {
    "name": "CoRoT-2b",
    "transitDepth": 2.750,
    "radialVelocityAmp": 568,
    "orbitalPeriod": 1.7429975,
    "eccentricity": 0,
    "star":{
        "effectiveTemperature": 5625,
        "stellarRatio_inSolarRatio": 0.902,
        "stellarLuminosity": -0.1305534,
        "stellarMass_inSolarMass": 0.970
    }
}

planet_3 = {
    "name": "HAT-P-14b",
    "transitDepth": 0.650,
    "radialVelocityAmp": 225.8,
    "orbitalPeriod": 4.6276820,
    "eccentricity": 0.1074,
    "star":{
        "effectiveTemperature": 6727.6100,
        "stellarRatio_inSolarRatio": 1.5452200,
        "stellarLuminosity": 0.6441211,
        "stellarMass_inSolarMass": 1.433000
    }
}

print('******************* Para o Planeta 1 *******************\n')
Mp1 = exoParams.planetMass_inJupMass(planet_1["radialVelocityAmp"], planet_1["orbitalPeriod"], planet_1["star"]["stellarMass_inSolarMass"])
a1AU = exoParams.semiMajorAxis_Kepler(planet_1["orbitalPeriod"], planet_1["star"]["stellarMass_inSolarMass"], Mp1)
rp1 = exoParams.planetRadius(planet_1["transitDepth"], planet_1["star"]["stellarRatio_inSolarRatio"])
print("Planeta:", planet_1["name"])
print('Grau de Insolação:', exoParams.radiationFlow(planet_1["star"]["stellarRatio_inSolarRatio"], planet_1["star"]["effectiveTemperature"], a1AU))
print('Temp. de Equilíbrio(K):', exoParams.equilibriumTemperature(planet_1["star"]["effectiveTemperature"], a1AU, planet_1["star"]["stellarRatio_inSolarRatio"]))
print('Raio do Planeta(RJup):', exoParams.planetRadius(planet_1["transitDepth"], planet_1["star"]["stellarRatio_inSolarRatio"]))
print("Massa do Planeta(MJup):", exoParams.planetMass_inJupMass(planet_1["radialVelocityAmp"], planet_1["orbitalPeriod"], planet_1["star"]["stellarMass_inSolarMass"]))
print("Semi-Eixo Maior(AU):", exoParams.semiMajorAxis_Kepler(planet_1["orbitalPeriod"], planet_1["star"]["stellarMass_inSolarMass"], Mp1))
print("Semi-Eixo Maior Bari(AU):", exoParams.semiMajorAxis_Bar(planet_1["orbitalPeriod"], planet_1["star"]["stellarMass_inSolarMass"], Mp1))
print("Prob de Transito(%):", exoParams.transitProbability(planet_1['star']['stellarRatio_inSolarRatio'], rp1, a1AU, planet_1['eccentricity'])*100)
print("Tempo de Transito(h):", exoParams.transitTime(planet_1["star"]["stellarRatio_inSolarRatio"], rp1, planet_1["orbitalPeriod"], a1AU))
print("Zonas de Habitabilidade(Inter, Cent, Exte)(AU):", exoParams.habitabilityZone(planet_1['star']['stellarLuminosity']))



print('\n******************* Para o Planeta 2*******************\n')
Mp2 = exoParams.planetMass_inJupMass(planet_2["radialVelocityAmp"], planet_2["orbitalPeriod"], planet_2["star"]["stellarMass_inSolarMass"])
a2AU = exoParams.semiMajorAxis_Kepler(planet_2["orbitalPeriod"], planet_2["star"]["stellarMass_inSolarMass"], Mp2)
rp2 = exoParams.planetRadius(planet_2["transitDepth"], planet_2["star"]["stellarRatio_inSolarRatio"])
print("Planeta:", planet_2["name"])
print('Grau de Insolação:', exoParams.radiationFlow(planet_2["star"]["stellarRatio_inSolarRatio"], planet_2["star"]["effectiveTemperature"], a2AU))
print('Raio do Planeta(RJup):', exoParams.planetRadius(planet_2["transitDepth"], planet_2["star"]["stellarRatio_inSolarRatio"]))
print("Massa do Planeta(MJup):", exoParams.planetMass_inJupMass(planet_2["radialVelocityAmp"], planet_2["orbitalPeriod"], planet_2["star"]["stellarMass_inSolarMass"]))
print("Semi-Eixo Maior(AU):", exoParams.semiMajorAxis_Kepler(planet_2["orbitalPeriod"], planet_2["star"]["stellarMass_inSolarMass"], Mp2))
print("Semi-Eixo Maior Bari(AU):", exoParams.semiMajorAxis_Bar(planet_2["orbitalPeriod"], planet_2["star"]["stellarMass_inSolarMass"], Mp2))
print("Prob de Transito(%):", exoParams.transitProbability(planet_2['star']['stellarRatio_inSolarRatio'], rp2, a2AU, planet_2['eccentricity'])*100)
print('Temp. de Equilíbrio(K):', exoParams.equilibriumTemperature(planet_2["star"]["effectiveTemperature"], a2AU, planet_2["star"]["stellarRatio_inSolarRatio"]))
print("Zonas de Habitabilidade(Inter, Cent, Exte)(AU):", exoParams.habitabilityZone(planet_2['star']['stellarLuminosity']))
print("Tempo de Transito(h):", exoParams.transitTime(planet_2["star"]["stellarRatio_inSolarRatio"], rp2, planet_2["orbitalPeriod"], a2AU))



print('\n******************* Para o Planeta 3 *******************\n')
Mp3 =  exoParams.planetMass_inJupMass(planet_3["radialVelocityAmp"], planet_3["orbitalPeriod"], planet_3["star"]["stellarMass_inSolarMass"])
a3AU = exoParams.semiMajorAxis_Kepler(planet_3["orbitalPeriod"], planet_3["star"]["stellarMass_inSolarMass"], Mp3)
rp3 = exoParams.planetRadius(planet_3["transitDepth"], planet_3["star"]["stellarRatio_inSolarRatio"])
print("Planeta:", planet_3["name"])
print('Grau de Insolação:', exoParams.radiationFlow(planet_3["star"]["stellarRatio_inSolarRatio"], planet_3["star"]["effectiveTemperature"], a3AU))
print('Temp. de Equilíbrio(K):', exoParams.equilibriumTemperature(planet_3["star"]["effectiveTemperature"], a3AU, planet_3["star"]["stellarRatio_inSolarRatio"]))
print('Raio do Planeta(RJup):', exoParams.planetRadius(planet_3["transitDepth"], planet_3["star"]["stellarRatio_inSolarRatio"]))
print("Massa do Planeta(MJup):", exoParams.planetMass_inJupMass(planet_3["radialVelocityAmp"], planet_3["orbitalPeriod"], planet_3["star"]["stellarMass_inSolarMass"]))
print("Semi-Eixo Maior(AU):", exoParams.semiMajorAxis_Kepler(planet_3["orbitalPeriod"], planet_3["star"]["stellarMass_inSolarMass"], Mp3))
print("Semi-Eixo Maior Bari(AU):", exoParams.semiMajorAxis_Bar(planet_3["orbitalPeriod"], planet_3["star"]["stellarMass_inSolarMass"], Mp3))
print("Prob de Transito(%):", exoParams.transitProbability(planet_3['star']['stellarRatio_inSolarRatio'], rp3, a3AU, planet_3['eccentricity'])*100)
print("Zonas de Habitabilidade(Inter, Cent, Exte)(AU):", exoParams.habitabilityZone(planet_3['star']['stellarLuminosity']))
print("Tempo de Transito(h):", exoParams.transitTime(planet_3["star"]["stellarRatio_inSolarRatio"], rp3, planet_3["orbitalPeriod"], a3AU))


(3.0, 4.0, 7.08)
******************* Para o Planeta 1 *******************

Planeta: BD+20 594b
Grau de Insolação: 25.587429681964664
Temp. de Equilíbrio(K): 6565.3680180430565
Raio do Planeta(RJup): 0.2822478982783838
Massa do Planeta(MJup): 0.05306631663384338
Semi-Eixo Maior(AU): 0.23555606417238928
Semi-Eixo Maior Bari(AU): 1.1891915087641184e-05
Prob de Transito(%): 2.457153357840919
Tempo de Transito(h): 7.826275148133799
Zonas de Habitabilidade(Inter, Cent, Exte)(AU): (0.017722535594746798, 0.0236300474596624, 0.041825184003602446)

******************* Para o Planeta 2*******************

Planeta: CoRoT-2b
Grau de Insolação: 926.748896600103
Raio do Planeta(RJup): 1.5320065727665793
Massa do Planeta(MJup): 3.300241985684898
Semi-Eixo Maior(AU): 0.028090910575148915
Semi-Eixo Maior Bari(AU): 9.090621479987927e-05
Prob de Transito(%): 14.930920467726091
Temp. de Equilíbrio(K): 5561.9706538156115
Zonas de Habitabilidade(Inter, Cent, Exte)(AU): (0.012783142688669997, 0.01704419025155